In [5]:
using BenchmarkTools: @btime
import SpecialPolynomials as sp
import PyPlot as plt

In [6]:
struct HO{T, V}
    hermite::T
    ω::Float64
    factors::V
    function HO(l, ω)
        factors = [(ω / π)^0.25 / √(2^n * factorial(n)) for n in 0:l-1]
        hermite = [sp.basis(sp.Hermite, n) for n in 0:l-1]
        return new{typeof(hermite), typeof(factors)}(hermite, ω, factors)
    end
end

In [7]:
ω = 0.25
basis = HO(8, ω);

In [8]:
function ho(n, x, basis)
    ξ = √basis.ω * x
    basis.factors[n+1] * basis.hermite[n+1](ξ) * exp(-ξ^2 / 2)
end

ho (generic function with 1 method)

In [9]:
@btime ho(3, 0.3, basis)

  89.917 ns (6 allocations: 96 bytes)


-0.1344001761701424

In [10]:
H = [sp.basis(sp.Hermite, i) for i in 0:30]

n = 3 + 1
H[n + 1](3.0)

876.0

In [11]:
function ho(n, x, H)
    x = √ω * x
    (ω / π)^0.25 / √(2^n * factorial(n)) * H(x) * exp(-x^2 / 2)
end

ho (generic function with 1 method)

In [12]:
@btime ho(3, 0.3, H[4])

  291.051 ns (14 allocations: 224 bytes)


-0.1344001761701424

In [ ]:
function E_n(n)
    return (n + 1/2) * ω
end

In [ ]:
x = [i for i in -10:20/2000:10];

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(x, 0.5 .* ω.^2 .* x.^2, lw = 1)
for n in 0:9
    plt.plot(x, E_n(n) .+ ho.(n, x, H[n + 1]).^2, lw = 1, label = "\$\\psi_{$n}\$")
end
plt.xlabel("x")
plt.ylabel("Energy + \$|ψ_{μ}|^2\$")
plt.title("Harmonic oscillator basis functions")
plt.grid()
plt.legend();